In [1]:
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta



In [2]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
weekday_stop_time_fixed =  pd.read_csv(r'weekday_stop_time_fixed.csv', sep=",", engine='python')

# route_short_name == the bus number
# the trip_id can be linked with the trip id in stop times
# trip id is unique to each row. 
trips =  pd.read_csv(r'C:\\Users\\이진우\\cs_summer\\Jin_project\\mmt_gtfs\\trips.txt', sep=",", engine='python')


In [34]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
trip_table_fixed =  pd.read_csv(r'trip_table_fixed.csv', sep=",", engine='python')

In [35]:
#select only the 2 hour time frame of 7-9 am from trip_table_fixed
trip_table_fixed = trip_table_fixed[trip_table_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]

In [36]:
#create graph
graph = {}
unique_table=trip_table_fixed.groupby(['full_bus_name','trip_id', 'stop_id_time']).size().reset_index()
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    graph[bus_name] = {}
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    bus_trip_id = unique_table.loc[index, 'trip_id']
    stop_id = unique_table.loc[index, 'stop_id_time']    
    graph[bus_name][bus_trip_id] = set([])

    


In [37]:
graph_ready = copy.deepcopy(graph)

In [38]:
#create graph form 
#bus#:set([bus_stops])
for index, row in trip_table_fixed.iterrows():
    route_name = row['full_bus_name']
    stop_id = row['stop_id_time']
    trip_id = row['trip_id']
    try:
        graph_ready[route_name][trip_id].add(stop_id)
    except:
        print(route_name, stop_id, trip_id)
        break

In [39]:
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
for key in graph_ready:
    for trip_id in graph_ready[key]:                   
        graph_temp = sorted(list(graph_ready[key][trip_id]), key=lambda x: int(x.split('_')[1]))
        graph_ready[key][trip_id] = graph_temp


In [40]:
for key in graph_ready:
    if 'DROP OFF' in key:
        print(key)

10_DROP OFFS ONLY
11_DROP OFFS ONLY
12_DROP OFFS ONLY
13_DROP OFFS ONLY
14_DROP OFFS ONLY
15_DROP OFFS ONLY
16_DROP OFFS ONLY
17_DROP OFFS ONLY
18_DROP OFFS ONLY
19_DROP OFFS ONLY
20_DROP OFFS ONLY
21_DROP OFFS ONLY
22_DROP OFFS ONLY
23_DROP OFFS ONLY
25_DROP OFFS ONLY
26_DROP OFFS ONLY
27_DROP OFFS ONLY
28_DROP OFFS ONLY
29_DROP OFFS ONLY
2_DROP OFFS ONLY
30_DROP OFFS ONLY
31_DROP OFFS ONLY
33_DROP OFFS ONLY
34_DROP OFFS ONLY
35_DROP OFFS ONLY
36_DROP OFFS ONLY
37_DROP OFFS ONLY
38_DROP OFFS ONLY
39_DROP OFFS ONLY
3_DROP OFFS ONLY
40_DROP OFFS ONLY
44_DROP OFFS ONLY
47_DROP OFFS ONLY
48_DROP OFFS ONLY
49_DROP OFFS ONLY
4_DROP OFFS ONLY
50_DROP OFFS ONLY
52_DROP OFFS ONLY
55_DROP OFFS ONLY
56_DROP OFFS ONLY
57_DROP OFFS ONLY
58_DROP OFFS ONLY
5_DROP OFFS ONLY
67_DROP OFFS ONLY
6_DROP OFFS ONLY
71_DROP OFFS ONLY
72_DROP OFFS ONLY
73_DROP OFFS ONLY
75_DROP OFFS ONLY
80_DROP OFFS ONLY


In [41]:
#star_stop will be a stop with a time in the morning between 7-8 AM 
def dfs(graph, start_stop):
     #bus_key = full_bus_name
    stop = start_stop
    visited, stack = set(), [stop] #creates the empty list and empty stack to go throu the while loop
    
    while stack:
        vertex = stack.pop() #takes out one of the values from stack
        current_time = int(vertex.split('_')[1]) #current time of the stop hit
        
        #if visited list is empty, this must be the first hit. 
        #Get all the stops that has the stop hits.
        #add the next stops to the stack. 
        if not visited:
            begin_time = int(stop.split('_')[1])
            visited.add(vertex)
            for bus_name in graph:
#                 #buses that drops off only cannot be a starting point
#                 if 'DROP OFF' in bus_name:
#                     continue
                for trip_id in graph[bus_name]:
                    try:
                        index_of_stop = graph[bus_name][trip_id].index(vertex)
                        next_stop = graph[bus_name][trip_id][index_of_stop+1]
                        stack.extend([next_stop])
                    except: 
                        continue
        else:          
            #checks if the time has passed 1 hour.
            #if it is more than 1 hour, simply continue to the next node. 
            if current_time - begin_time < 3600:
                if vertex not in visited:
                    visited.add(vertex)
                    for bus_name in graph:
                        for trip_id in graph[bus_name]:
#                             if 'DROP OFF' in bus_name:
#                                 try:
#                                     #check if the previous stop was the same trip id.
#                                     #if it is, on the same bus so drop off only is fine
#                                     stop_id=drop_off_only[trip_id]
#                                     if stop_id == vertex:
#                                         pass
#                                     else: 
#                                         continue
#                                 except: 
#                                     continue
                            try:
                                index_of_stop = graph[bus_name][trip_id].index(vertex)
                                next_stop = graph[bus_name][trip_id][index_of_stop+1]
                                stack.extend([next_stop])
#                                 if 'DROP OFF' in bus_name:
#                                     drop_off_only[trip_id] = vertex
                            except: 
                                continue
            else: 
                continue 
    return visited



In [42]:
list_of_stops = trip_table_fixed['stop_id_time'].unique()
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
list_of_stops_fixed = sorted(list_of_stops, key=lambda x: int(x.split('_')[1]))


In [ ]:
dic_of_rows = {}
for start_stop in list_of_stops_fixed:
    dic_of_rows[start_stop]=[]
count = 0
#choose a stop to first begin
for start_stop in list_of_stops_fixed:
    list_of_stop=dfs(graph_ready, start_stop)
    dic_of_rows[start_stop].append(list_of_stop)
    print('starting stop is ', start_stop)
#     if count == 5:
#         break
#     count += 1
    



starting stop is  6100_25200
starting stop is  5100_25200
starting stop is  1878_25200
starting stop is  4100_25200
starting stop is  2200_25200
starting stop is  3196_25200
starting stop is  1101_25200
starting stop is  1486_25200
starting stop is  6350_25200
starting stop is  6476_25200
starting stop is  4401_25200
starting stop is  1100_25200
starting stop is  10_25200
starting stop is  2195_25200
starting stop is  9800_25200
starting stop is  3841_25201
starting stop is  7991_25201
starting stop is  1410_25202
starting stop is  435_25202
starting stop is  2923_25203
starting stop is  435_25204
starting stop is  9996_25204
starting stop is  758_25204
starting stop is  2885_25205
starting stop is  445_25206
starting stop is  6539_25206
starting stop is  8869_25206
starting stop is  2146_25207
starting stop is  1579_25207
starting stop is  5236_25207
starting stop is  8869_25207
starting stop is  5715_25208
starting stop is  1968_25208
starting stop is  7723_25208
starting stop is  18

starting stop is  8297_25320
starting stop is  10_25320
starting stop is  5100_25320
starting stop is  133_25320
starting stop is  1505_25320
starting stop is  3301_25320
starting stop is  668_25323
starting stop is  2270_25323
starting stop is  839_25323
starting stop is  8333_25323
starting stop is  8333_25324
starting stop is  2091_25325
starting stop is  499_25327
starting stop is  9192_25327
starting stop is  1579_25327
starting stop is  688_25327
starting stop is  971_25328
starting stop is  703_25329
starting stop is  9984_25330
starting stop is  1462_25330
starting stop is  2620_25331
starting stop is  1462_25331
starting stop is  2902_25332
starting stop is  2161_25332
starting stop is  8418_25332
starting stop is  4192_25332
starting stop is  2401_25332
starting stop is  1309_25332
starting stop is  2323_25333
starting stop is  941_25333
starting stop is  7840_25333
starting stop is  9154_25333
starting stop is  9725_25334
starting stop is  9154_25334
starting stop is  2383_2

In [ ]:
dic_of_rows